<div style="font-size:18pt; padding-top:20px; text-align:center">СЕМИНАР. <b>Деревья решений и </b> <span style="font-weight:bold; color:green">NumPy/SciPy/Sklearn</span></div><hr>
<div style="text-align:right;">Папулин С.Ю. <span style="font-style: italic;font-weight: bold;">(papulin.study@yandex.ru)</span></div>

<a name="0"></a>
<div><span style="font-size:14pt; font-weight:bold">Содержание</span>
    <ol>
        <li><a href="#1">Классификация</a></li>
        <li><a href="#2">Регрессия</a></li>
        <li><a href="#3">Источники</a>
        </li>
    </ol>
</div>

<p><b>Подключение библиотек</b></p>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.datasets import make_classification
from matplotlib import cm
from matplotlib.colors import ListedColormap

<a name="1"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">1. Классификация</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<a href="http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html">DecisionTreeClassifier</a>

In [ ]:
from sklearn import tree

<p><b>Исходные данные</b></p>

In [ ]:
n = 100
x, y = make_classification(n_samples=n, n_features=2, n_redundant=0, 
                           n_informative=2, n_clusters_per_class=1, n_classes=2, class_sep=0.5,
                           random_state=123)

clrMap = ListedColormap(["blue", "red"])
plt.scatter(x[:,0], x[:,1], c=y, cmap=clrMap)
plt.grid(True)
plt.show()

<p><b>Формирование обучающего и тестового подмножеств</b></p>

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

<p><b>Обучение</b></p>

In [ ]:
decTree = tree.DecisionTreeClassifier(criterion="entropy")

In [ ]:
decTree.fit(x_train, y_train)

In [ ]:
plt.scatter(x_train[:,0], x_train[:,1], c=y_train, s=80, cmap=clrMap, alpha=0.5)
plt.scatter(x_train[:, 0], x_train[:, 1], c=decTree.predict(x_train), s=20, cmap=clrMap)
plt.grid(True)
plt.show()

<p><b>Проверка на тестовом подмножестве</b></p>

In [ ]:
score = decTree.score(x_test, y_test)
score

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
accuracy_score(y_test, decTree.predict(x_test))

In [ ]:
print(classification_report(y_test, decTree.predict(x_test), target_names=["Class 0", "Class 1"]))

In [ ]:
plt.scatter(x_test[:,0], x_test[:,1], c=y_test, s=80, cmap=clrMap, alpha=0.5)
plt.scatter(x_test[:, 0], x_test[:, 1], c=decTree.predict(x_test), s=20, cmap=clrMap)
plt.grid(True)
plt.show()

<p><b>Графики</b></p>

In [ ]:
step = 0.01
xx, yy = np.meshgrid(np.arange(x[:,0].min(), x[:,0].max(), step), np.arange(x[:,1].min(), x[:,1].max(), step))

In [ ]:
points = np.c_[xx.ravel(), yy.ravel()]
Z = decTree.predict(points)
Z = Z.reshape(xx.shape)

In [ ]:
plt.figure(1, figsize=[12, 4])

plt.subplot(1,2,1)
plt.title("Train data")
plt.contourf(xx, yy, Z, cmap=clrMap, alpha=.5)
plt.scatter(x_train[:,0], x_train[:,1], c=y_train, s=80, cmap=clrMap, alpha=0.5)
plt.scatter(x_train[:, 0], x_train[:, 1], c=decTree.predict(x_train), s=20, cmap=clrMap)
plt.grid(True)

plt.subplot(1,2,2)
plt.title("Test data")
plt.contourf(xx, yy, Z, cmap=clrMap, alpha=.5)
plt.scatter(x_test[:,0], x_test[:,1], c=y_test, s=80, cmap=clrMap, alpha=0.5)
plt.scatter(x_test[:,0], x_test[:,1], c=decTree.predict(x_test), s=20, cmap=clrMap)
plt.grid(True)

plt.show()

<p><b>Структура Tree</b></p>

In [ ]:
decTree.tree_

In [ ]:
decTree.tree_.children_left

In [ ]:
decTree.tree_.children_right

In [ ]:
decTree.tree_.threshold

In [ ]:
decTree.tree_.feature

In [ ]:
decTree.tree_.value

In [ ]:
#http://stackoverflow.com/questions/20224526/how-to-extract-the-decision-rules-from-scikit-learn-decision-tree
def get_code(tree, feature_names):
    left = tree.tree_.children_left
    right = tree.tree_.children_right
    threshold = tree.tree_.threshold
    features = [feature_names[i] for i in tree.tree_.feature]
    value = tree.tree_.value

    def recurse(left, right, threshold, features, node):
        if (threshold[node] != -2):
            print("if ( " + features[node] + " <= " + str(threshold[node]) + " ) {")
            if left[node] != -1:
                recurse (left, right, threshold, features,left[node])
            print("} else {")
            if right[node] != -1:
                recurse (left, right, threshold, features,right[node])
            print("}")
        else:
            print("return " + str(value[node]))

    recurse(left, right, threshold, features, 0)

In [ ]:
get_code(decTree, ["X0", "X1"])

<p><b>Три класса</b></p>

<p><b>Исходные данные</b></p>

In [ ]:
n = 100
x, y = make_classification(n_samples=n, n_features=2, n_redundant=0, 
                           n_informative=2, n_clusters_per_class=1, n_classes=3, class_sep=1,
                           random_state=1234)

In [ ]:
clrMap = ListedColormap(["blue", "red", "green"])
plt.scatter(x[:,0], x[:,1], c=y, cmap=clrMap)
plt.grid(True)
plt.show()

<p><b>Формирование обучающего и тестового подмножеств</b></p>

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

<p><b>Обучение</b></p>

In [ ]:
decTree = tree.DecisionTreeClassifier(criterion="entropy")

In [ ]:
decTree.fit(x_train, y_train)

In [ ]:
plt.scatter(x_train[:,0], x_train[:,1], c=y_train, s=80, cmap=clrMap, alpha=0.5)
plt.scatter(x_train[:, 0], x_train[:, 1], c=decTree.predict(x_train), s=20, cmap=clrMap)
plt.grid(True)
plt.show()

<p><b>Проверка на тестовом подмножестве</b></p>

In [ ]:
score = decTree.score(x_test, y_test)
score

In [ ]:
print(classification_report(y_test, decTree.predict(x_test), target_names=["Class 0", "Class 1", "Class 2"]))

<p><b>Графики</b></p>

In [ ]:
step = 0.01
xx, yy = np.meshgrid(np.arange(x[:,0].min(), x[:,0].max(), step), np.arange(x[:,1].min(), x[:,1].max(), step))

In [ ]:
points = np.c_[xx.ravel(), yy.ravel()]
#Z = decTree.predict_proba(points)[:, 1]
Z = decTree.predict(points)
Z = Z.reshape(xx.shape)

In [ ]:
plt.figure(1, figsize=[12, 4])

plt.subplot(1,2,1)
plt.title("Train data")
plt.contourf(xx, yy, Z, cmap=clrMap, alpha=.5)
plt.scatter(x_train[:,0], x_train[:,1], c=y_train, s=80, cmap=clrMap, alpha=0.5)
plt.scatter(x_train[:, 0], x_train[:, 1], c=decTree.predict(x_train), s=20, cmap=clrMap)
plt.grid(True)

plt.subplot(1,2,2)
plt.title("Test data")
plt.contourf(xx, yy, Z, cmap=clrMap, alpha=.5)
plt.scatter(x_test[:,0], x_test[:,1], c=y_test, s=80, cmap=clrMap, alpha=0.5)
plt.scatter(x_test[:,0], x_test[:,1], c=decTree.predict(x_test), s=20, cmap=clrMap)
plt.grid(True)

plt.show()

<a name="2"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">2. Регрессия</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<a href="http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html">DecisionTreeRegressor</a>

In [ ]:
from scipy import stats

<p><b>Исходные данные</b></p>

In [ ]:
n = 100
x = stats.uniform.rvs(size=n, loc=4, scale=8)

In [ ]:
y = 2 + 0.3*x

In [ ]:
mu = 0
sigma = 0.5
y1 = stats.norm.rvs(size=n, loc=mu, scale=sigma) + y

In [ ]:
plt.title("Initial data")
plt.plot(x, y1, "o")
plt.grid(True)
plt.show()

<p><b>Формирование обучающего и тестового подмножеств</b></p>

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y1, test_size=0.3, random_state=0)

In [ ]:
x_train = x_train[:, np.newaxis]
x_test = x_test[:, np.newaxis]

<p><b>Обучение</b></p>

In [ ]:
decTreeReg = tree.DecisionTreeRegressor(criterion="mse", random_state=0)

In [ ]:
decTreeReg.fit(x_train, y_train)

<p><b>Проверка на тестовом подмножестве</b></p>

In [ ]:
decTreeReg.score(x_test, y_test)

<p><b>Графики</b></p>

In [ ]:
xx = np.arange(x.min(), x.max(), 0.01)[:, np.newaxis]

In [ ]:
plt.plot(xx, decTreeReg.predict(xx), c="r", label="max_depth=5", linewidth=2)
plt.plot(x_test, y_test, "o")
plt.vlines(x_test, ymin=y_test, ymax=decTreeReg.predict(x_test), colors="black", linestyles="dotted")
plt.plot(x_test, decTreeReg.predict(x_test), "o", color="red", lw=2)
plt.grid()
plt.show()